In [1]:
import cv2
import numpy as np

In [5]:
def timeVideo(in_add, out_add):
    
    cap = cv2.VideoCapture(in_add)
    ret, frame = cap.read()
    count = 1
    
    while(cap.isOpened()):
        if ret:
            cv2.imwrite(out_add + "/frame{}.png".format(count), frame)
            count += 1

        else:
            break
        ret, frame = cap.read()

    cap.release()
    cv2.destroyAllWindows()
    print(count)

In [6]:
timeVideo("./data/test.mp4", "./data/frames_test")

10799
